In [ ]:
import os
import numpy as np
import tensorflow as tf
import streamlit as st
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# ---- STEP 1: DATA PREPROCESSING ----
dataset_path = r"D:\Project\Bolt ,Nut, project\blnw-images-224"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_data.class_indices.keys())

# ---- STEP 2: CNN MODEL ----
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(len(class_names), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- STEP 3: TRAIN & SAVE MODEL ----
model = create_cnn_model()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

model.save("blnw_cnn_model.h5")
print("Model saved successfully!")

# ---- STEP 4: STREAMLIT GUI ----
st.title("Mechanical Part Detector (Bolt, Nut, Washer, Locating Pin)")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    image = load_img(uploaded_file, target_size=(224, 224))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    # Load Model
    model = tf.keras.models.load_model("blnw_cnn_model.h5")

    prediction = model.predict(image_array)
    predicted_class = class_names[np.argmax(prediction)]

    st.image(uploaded_file, caption=f"Predicted: {predicted_class}", use_column_width=True)

Found 12186 images belonging to 4 classes.
Found 3044 images belonging to 4 classes.


C:\Users\MSVV\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\MSVV\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 704s 2s/step - accuracy: 0.7606 - loss: 5.1605 - val_accuracy: 0.2500 - val_loss: 75.8761
Epoch 2/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.8912 - loss: 0.3209 - val_accuracy: 0.8219 - val_loss: 2.3288
Epoch 3/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 606s 2s/step - accuracy: 0.9140 - loss: 0.2385 - val_accuracy: 0.6239 - val_loss: 33.2296
Epoch 4/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 612s 2s/step - accuracy: 0.9115 - loss: 0.2250 - val_accuracy: 0.5762 - val_loss: 23.0060
Epoch 5/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 1851s 5s/step - accuracy: 0.9237 - loss: 0.2096 - val_accuracy: 0.7244 - val_loss: 3.8086
Epoch 6/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 607s 2s/step - accuracy: 0.9411 - loss: 0.1572 - val_accuracy: 0.9350 - val_loss: 1.8088
Epoch 7/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 599s 2s/step - accuracy: 0.9467 - loss: 0.1566 - val_accuracy: 0.6567 - val_loss: 38.6627
Epoch 8/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 597s 2s/step - accuracy: 0.9371 - loss: 0.1777 - val_accuracy: 

Model saved successfully!


2025-02-27 20:32:59.133 
  command:

    streamlit run C:\Users\MSVV\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [5]:
model.save("blnw_cnn_model.h5")


In [7]:
model.save(r"D:\blnw_detection\blnw_cnn_model.h5")


In [9]:
import os
save_path = r"D:\blnw_detection"
os.makedirs(save_path, exist_ok=True)  # Create the directory if it doesn't exist

model.save(os.path.join(save_path, "blnw_cnn_model.h5"))


In [17]:
import cv2
import numpy as np
import tensorflow as tf
import streamlit as st
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# ---- Load the trained model ----
MODEL_PATH = r"D:\blnw_detection\blnw_cnn_model.h5"  # Update with your saved model
model = tf.keras.models.load_model(MODEL_PATH)

# Define class labels (must match dataset)
class_names = ['bolt', 'locatingpin', 'nut', 'washer']

# ---- Streamlit UI ----
st.title("Mechanical Parts Detector 🔧")
st.sidebar.title("Upload Image or Use Webcam")
choice = st.sidebar.radio("Select Input Mode", ("Upload Image", "Use Webcam"))

def predict_image(image):
    """Preprocess the image and get predictions."""
    img_array = img_to_array(image) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    
    return predicted_class, confidence

# ---- Image Upload Mode ----
if choice == "Upload Image":
    uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file).resize((224, 224))
        st.image(image, caption="Uploaded Image", use_container_width=True)
        
        predicted_class, confidence = predict_image(image)
        st.success(f"Detected: **{predicted_class}** ({confidence:.2f}%)")

# ---- Webcam Mode ----
elif choice == "Use Webcam":
    st.write("Starting webcam... Press `Q` to exit.")

    cap = cv2.VideoCapture(0)  # Open laptop camera
    frame_placeholder = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture video!")
            break

        # Convert frame to RGB (Streamlit uses RGB, OpenCV uses BGR)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(frame_rgb, (224, 224))

        # Predict object
        predicted_class, confidence = predict_image(img_resized)

        # Convert to grayscale for edge detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 20 and h > 20:  # Ignore small noise
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                size_text = f"Size: {w}x{h} px"
                cv2.putText(frame, size_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show prediction
        text = f"{predicted_class} ({confidence:.2f}%)"
        cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Display video stream in Streamlit
        frame_placeholder.image(frame, channels="BGR")

        # Stop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import streamlit as st
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

# ---- STEP 1: DATA PREPROCESSING ----
dataset_path = r"D:\Project\Bolt ,Nut, project\blnw-images-224"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_data.class_indices.keys())  # Get class names
print("Class Names:", class_names)

# Compute class weights to fix imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# ---- STEP 2: CNN MODEL CREATION ----
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2,2),
        
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(class_names), activation='softmax')  # Output layer
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train model
model = create_cnn_model()
history = model.fit(train_data, validation_data=val_data, epochs=5, class_weight=class_weights_dict)

# ---- STEP 3: SAVE MODEL ----
save_path = r"D:\blnw_detection\blnw_cnn_model.h5"
model.save(save_path)
print(f"Model saved at {save_path}")

# ---- STEP 4: STREAMLIT GUI ----
st.title("Mechanical Parts Detector 🔧")
st.sidebar.title("Upload Image or Use Webcam")
choice = st.sidebar.radio("Select Input Mode", ("Upload Image", "Use Webcam"))

# Load the trained model
model = tf.keras.models.load_model(save_path)

def predict_image(image):
    """Preprocess the image and get predictions."""
    image = image.resize((224, 224))  # Resize to match training input
    img_array = img_to_array(image)
    img_array = img_array / 255.0  # Normalize (same as training)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    
    return predicted_class, confidence

# ---- Image Upload Mode ----
if choice == "Upload Image":
    uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file).resize((224, 224))
        st.image(image, caption="Uploaded Image", use_container_width=True)
        
        predicted_class, confidence = predict_image(image)
        st.success(f"Detected: **{predicted_class}** ({confidence:.2f}%)")

# ---- Webcam Mode ----
elif choice == "Use Webcam":
    st.write("Starting webcam... Press `Q` to exit.")

    cap = cv2.VideoCapture(0)
    frame_placeholder = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture video!")
            break

        # Convert OpenCV BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)  # Convert to PIL Image

        # Resize and predict
        predicted_class, confidence = predict_image(frame_pil)

        # Show results
        text = f"{predicted_class} ({confidence:.2f}%)"
        cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        frame_placeholder.image(frame, channels="BGR")


Found 12186 images belonging to 4 classes.
Found 3044 images belonging to 4 classes.
Class Names: ['bolt', 'locatingpin', 'nut', 'washer']


C:\Users\MSVV\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


C:\Users\MSVV\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 307s 789ms/step - accuracy: 0.7322 - loss: 0.8293 - val_accuracy: 0.8614 - val_loss: 0.6483
Epoch 2/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 295s 774ms/step - accuracy: 0.9615 - loss: 0.1177 - val_accuracy: 0.9185 - val_loss: 0.5131
Epoch 3/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 293s 767ms/step - accuracy: 0.9763 - loss: 0.0634 - val_accuracy: 0.9060 - val_loss: 0.7171
Epoch 4/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 293s 768ms/step - accuracy: 0.9864 - loss: 0.0422 - val_accuracy: 0.8890 - val_loss: 1.0132
Epoch 5/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 295s 772ms/step - accuracy: 0.9888 - loss: 0.0307 - val_accuracy: 0.9014 - val_loss: 0.8600


Model saved at D:\blnw_detection\blnw_cnn_model.h5


In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Load Trained Model
MODEL_PATH = r"D:\blnw_detection\blnw_cnn_model.h5"
model = tf.keras.models.load_model(MODEL_PATH)

# Class Names (should match dataset labels)
class_names = ['bolt', 'locatingpin', 'nut', 'washer']

def predict_image(image):
    """Preprocess the image and get predictions."""
    image = image.resize((224, 224))  # Resize image
    img_array = img_to_array(image)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    
    return predicted_class, confidence


In [27]:
import cv2
import streamlit as st
from PIL import Image
from predict import predict_image  # Import prediction function

st.title("Mechanical Parts Detector 🔧")
st.sidebar.title("Upload Image or Use Webcam")
choice = st.sidebar.radio("Select Input Mode", ("Upload Image", "Use Webcam"))

# ---- IMAGE UPLOAD MODE ----
if choice == "Upload Image":
    uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file).resize((224, 224))
        st.image(image, caption="Uploaded Image", use_container_width=True)
        
        predicted_class, confidence = predict_image(image)
        st.success(f"Detected: **{predicted_class}** ({confidence:.2f}%)")

# ---- WEBCAM MODE ----
elif choice == "Use Webcam":
    st.write("Starting webcam... Press `Q` to exit.")

    cap = cv2.VideoCapture(0)
    frame_placeholder = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture video!")
            break

        # Convert OpenCV BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)  # Convert to PIL Image

        # Resize and predict
        predicted_class, confidence = predict_image(frame_pil)

        # Show results
        text = f"{predicted_class} ({confidence:.2f}%)"
        cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        frame_placeholder.image(frame, channels="BGR")


ModuleNotFoundError: No module named 'predict'

In [29]:
import os
save_path = r"D:\BLNW"
os.makedirs(save_path, exist_ok=True)  # Create folder if it doesn't exist

model.save(os.path.join(save_path, "blnw_cnn_model.h5"))


In [31]:
import cv2
import numpy as np
import tensorflow as tf
import streamlit as st
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image

# ---- Load the trained model ----
MODEL_PATH = r"D:\BLNW\blnw_cnn_model.h5"  # Ensure this path is correct
model = tf.keras.models.load_model(MODEL_PATH)

# Define class labels (must match dataset)
class_names = ['bolt', 'locatingpin', 'nut', 'washer']

# ---- Streamlit UI ----
st.title("Mechanical Parts Detector 🔧")
st.sidebar.title("Upload Image or Use Webcam")
choice = st.sidebar.radio("Select Input Mode", ("Upload Image", "Use Webcam"))

def predict_image(image):
    """Preprocess the image and get predictions."""
    img_array = img_to_array(image)
    img_array = cv2.resize(img_array, (224, 224))  # Ensure correct input size
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    
    return predicted_class, confidence

# ---- Image Upload Mode ----
if choice == "Upload Image":
    uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("RGB")
        image = image.resize((224, 224))  # Ensure resizing
        st.image(image, caption="Uploaded Image", use_column_width=True)
        
        predicted_class, confidence = predict_image(image)
        st.success(f"Detected: **{predicted_class}** ({confidence:.2f}%)")

# ---- Webcam Mode ----
elif choice == "Use Webcam":
    st.write("Starting webcam... Press `Q` to exit.")

    cap = cv2.VideoCapture(0)  # Open laptop camera
    frame_placeholder = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture video!")
            break

        # Convert frame to RGB (Streamlit uses RGB, OpenCV uses BGR)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(frame_rgb, (224, 224))

        # Predict object
        predicted_class, confidence = predict_image(img_resized)

        # Draw bounding box around detected parts
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 20 and h > 20:  # Ignore small noise
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                size_text = f"Size: {w}x{h} px"
                cv2.putText(frame, size_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show prediction
        text = f"{predicted_class} ({confidence:.2f}%)"
        cv2.putText(frame, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Display video stream in Streamlit
        frame_placeholder.image(frame, channels="BGR")

        # Stop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
model = tf.keras.models.load_model("blnw_cnn_model.h5")


In [3]:
# Get true labels and predicted labels
y_true = val_data.classes  # True labels
y_pred_probs = model.predict(val_data)  # Get predicted probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels


NameError: name 'val_data' is not defined

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path and image size
dataset_path = r"blnw-images-224"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Reload validation dataset
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Important: No shuffling to align predictions with true labels
)

class_names = list(val_data.class_indices.keys())  # Get class names


Found 3044 images belonging to 4 classes.
